<a href="https://colab.research.google.com/github/chetnalshapur/Development-of-Interactive-Cyber-Threat-Visualization-Dashboard/blob/main/SQL_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Visualize Query 2: Bar Chart of Total Attacks by Type

This bar chart shows the distribution of different attack types, making it easy to identify the most frequent threats. Hover over each bar to see the exact count for that attack type.

In [ ]:
import plotly.express as px

# Create an interactive bar chart
fig = px.bar(
    df_attacks_by_type,
    x='attack_type',
    y='total_attacks',
    title='Total Attacks by Type',
    labels={'attack_type': 'Attack Type', 'total_attacks': 'Total Attacks'},
    color='total_attacks', # Color bars based on total attacks
    color_continuous_scale=px.colors.sequential.Viridis # Choose a color scale
)

# Update layout for better readability
fig.update_layout(xaxis_title_standoff=25, yaxis_title_standoff=25)
fig.update_xaxes(tickangle=45)

# Show the interactive plot
fig.show()

### 6. Execute Query 2: Count total attacks by type

In [ ]:
# SQL Query 2: Count total attacks by type
query2 = """SELECT
    attack_type,
    COUNT(attack_id) AS total_attacks
FROM
    attack_details
GROUP BY
    attack_type
ORDER BY
    total_attacks DESC;
"""

# Execute the query and load results into a pandas DataFrame
df_attacks_by_type = pd.read_sql_query(query2, conn)

# Display the DataFrame
print("DataFrame for Query 2: Count Total Attacks by Type")
display(df_attacks_by_type)

print(f"\nTotal attack types: {len(df_attacks_by_type)}")

DataFrame for Query 2: Count Total Attacks by Type


,attack_type,total_attacks
0,Ping Flood,6
1,Brute Force,2
2,SQL Injection,1
3,DDoS,1



Total attack types: 4


### 7. Execute Query 3: Retrieve high-risk threats based on associated network activity and threat level

In [ ]:
# SQL Query 3: Retrieve high-risk threats based on associated network activity and threat level
query3 = """SELECT
    ad.attack_id,
    ad.attack_type,
    ad.detection_time,
    tl.level_name AS threat_level,
    COUNT(nl.log_id) AS associated_log_entries_count, -- Proxy for 'packet_rate'
    ad.description AS attack_description,
    ad.status
FROM
    attack_details ad
JOIN
    threat_levels tl ON ad.threat_level_id = tl.level_id
LEFT JOIN
    network_logs nl ON ad.log_id = nl.log_id
WHERE
    tl.level_name IN ('High', 'Critical')
GROUP BY
    ad.attack_id, ad.attack_type, ad.detection_time, tl.level_name, ad.description, ad.status
HAVING
    COUNT(nl.log_id) >= 1 -- Adjusted threshold to include attacks with at least one associated log entry
ORDER BY
    associated_log_entries_count DESC, ad.detection_time DESC;
"""

# Execute the query and load results into a pandas DataFrame
df_high_risk_threats = pd.read_sql_query(query3, conn)

# Display the DataFrame
print("DataFrame for Query 3: High-Risk Threats")
display(df_high_risk_threats)

print(f"\nTotal high-risk threats: {len(df_high_risk_threats)}")

DataFrame for Query 3: High-Risk Threats


,attack_id,attack_type,detection_time,threat_level,associated_log_entries_count,attack_description,status
0,10,Ping Flood,2023-10-26 10:00:17,High,1,Continued high volume of ICMP packets.,MITIGATED
1,9,Ping Flood,2023-10-26 10:00:16,High,1,Continued high volume of ICMP packets.,MITIGATED
2,8,Ping Flood,2023-10-26 10:00:15,High,1,Continued high volume of ICMP packets.,MITIGATED
3,7,Ping Flood,2023-10-26 10:00:14,High,1,Continued high volume of ICMP packets.,MITIGATED
4,6,Ping Flood,2023-10-26 10:00:13,High,1,Continued high volume of ICMP packets.,MITIGATED
5,5,Ping Flood,2023-10-26 10:00:11,High,1,High volume of ICMP packets.,MITIGATED
6,4,SQL Injection,2023-10-26 10:00:09,Critical,1,Attempt to inject malicious SQL code.,DETECTED
7,3,Brute Force,2023-10-26 10:00:06,High,1,More failed login attempts.,INVESTIGATING
8,2,Brute Force,2023-10-26 10:00:04,High,1,Repeated failed login attempts.,DETECTED
9,1,DDoS,2023-10-26 10:00:03,High,1,Multiple connection attempts from different so...,DETECTED



Total high-risk threats: 10


### 4. Setup Database and Load Sample Data

First, let's set up an in-memory SQLite database and populate it with some sample data so we can execute our SQL queries.

In [ ]:
import sqlite3
import pandas as pd

# Connect to an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create tables based on the schema
cursor.execute("""CREATE TABLE threat_levels (
    level_id INT PRIMARY KEY,
    level_name VARCHAR(50) NOT NULL UNIQUE,
    description TEXT
);
""")
cursor.execute("""CREATE TABLE network_logs (
    log_id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp DATETIME NOT NULL,
    source_ip VARCHAR(45),
    destination_ip VARCHAR(45),
    protocol VARCHAR(10),
    port INT,
    action VARCHAR(20),
    bytes_sent INT,
    bytes_received INT,
    event_type VARCHAR(50)
);
""")
cursor.execute("""CREATE TABLE attack_details (
    attack_id INTEGER PRIMARY KEY AUTOINCREMENT,
    log_id INT,
    attack_type VARCHAR(100) NOT NULL,
    detection_time DATETIME NOT NULL,
    threat_level_id INT,
    description TEXT,
    status VARCHAR(50),
    FOREIGN KEY (log_id) REFERENCES network_logs(log_id),
    FOREIGN KEY (threat_level_id) REFERENCES threat_levels(level_id)
);
""")

# Insert sample data into threat_levels
cursor.execute("INSERT INTO threat_levels (level_id, level_name, description) VALUES (1, 'Low', 'Minimal impact');")
cursor.execute("INSERT INTO threat_levels (level_id, level_name, description) VALUES (2, 'Medium', 'Moderate impact');")
cursor.execute("INSERT INTO threat_levels (level_id, level_name, description) VALUES (3, 'High', 'Significant impact');")
cursor.execute("INSERT INTO threat_levels (level_id, level_name, description) VALUES (4, 'Critical', 'Severe, widespread impact');")

# Insert sample data into network_logs
network_logs_data = [
    ('2023-10-26 10:00:01', '192.168.1.10', '10.0.0.1', 'TCP', 80, 'ALLOW', 1024, 512, 'CONNECTION'),
    ('2023-10-26 10:00:02', '192.168.1.11', '10.0.0.2', 'UDP', 53, 'ALLOW', 64, 128, 'DNS_QUERY'),
    ('2023-10-26 10:00:03', '192.168.1.12', '10.0.0.3', 'TCP', 443, 'DENY', 0, 0, 'CONNECTION_ATTEMPT'),
    ('2023-10-26 10:00:04', '192.168.1.13', '10.0.0.4', 'TCP', 22, 'ALERT', 256, 128, 'LOGIN_ATTEMPT'),
    ('2023-10-26 10:00:05', '192.168.1.10', '10.0.0.1', 'TCP', 80, 'ALLOW', 2048, 1024, 'CONNECTION'),
    ('2023-10-26 10:00:06', '192.168.1.13', '10.0.0.4', 'TCP', 22, 'ALERT', 512, 256, 'LOGIN_ATTEMPT'),
    ('2023-10-26 10:00:07', '192.168.1.10', '10.0.0.1', 'TCP', 80, 'ALLOW', 1500, 750, 'CONNECTION'),
    ('2023-10-26 10:00:08', '192.168.1.14', '10.0.0.5', 'UDP', 123, 'ALLOW', 100, 100, 'NTP_SYNC'),
    ('2023-10-26 10:00:09', '192.168.1.15', '10.0.0.6', 'TCP', 3306, 'DENY', 0, 0, 'SQL_ATTEMPT'),
    ('2023-10-26 10:00:10', '192.168.1.16', '10.0.0.7', 'TCP', 8080, 'ALLOW', 3000, 1500, 'HTTP_REQUEST'),
    ('2023-10-26 10:00:11', '192.168.1.17', '10.0.0.8', 'ICMP', None, 'ALERT', 64, 64, 'PING_SWEEP'),
    ('2023-10-26 10:00:12', '192.168.1.18', '10.0.0.9', 'TCP', 21, 'DENY', 0, 0, 'FTP_ACCESS'),
    ('2023-10-26 10:00:13', '192.168.1.17', '10.0.0.8', 'ICMP', None, 'ALERT', 64, 64, 'PING_SWEEP'),
    ('2023-10-26 10:00:14', '192.168.1.17', '10.0.0.8', 'ICMP', None, 'ALERT', 64, 64, 'PING_SWEEP'),
    ('2023-10-26 10:00:15', '192.168.1.17', '10.0.0.8', 'ICMP', None, 'ALERT', 64, 64, 'PING_SWEEP'),
    ('2023-10-26 10:00:16', '192.168.1.17', '10.0.0.8', 'ICMP', None, 'ALERT', 64, 64, 'PING_SWEEP'),
    ('2023-10-26 10:00:17', '192.168.1.17', '10.0.0.8', 'ICMP', None, 'ALERT', 64, 64, 'PING_SWEEP'),
    ('2023-10-26 10:00:18', '192.168.1.19', '10.0.0.10', 'TCP', 80, 'ALLOW', 1024, 512, 'CONNECTION')
]
cursor.executemany("INSERT INTO network_logs (timestamp, source_ip, destination_ip, protocol, port, action, bytes_sent, bytes_received, event_type) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)", network_logs_data)

# Insert sample data into attack_details (linking to network_logs and threat_levels)
attack_details_data = [
    (3, 'DDoS', '2023-10-26 10:00:03', 3, 'Multiple connection attempts from different sources.', 'DETECTED'), # log_id 3
    (4, 'Brute Force', '2023-10-26 10:00:04', 3, 'Repeated failed login attempts.', 'DETECTED'), # log_id 4
    (6, 'Brute Force', '2023-10-26 10:00:06', 3, 'More failed login attempts.', 'INVESTIGATING'), # log_id 6
    (9, 'SQL Injection', '2023-10-26 10:00:09', 4, 'Attempt to inject malicious SQL code.', 'DETECTED'), # log_id 9
    (11, 'Ping Flood', '2023-10-26 10:00:11', 3, 'High volume of ICMP packets.', 'MITIGATED'), # log_id 11
    (13, 'Ping Flood', '2023-10-26 10:00:13', 3, 'Continued high volume of ICMP packets.', 'MITIGATED'), # log_id 13
    (14, 'Ping Flood', '2023-10-26 10:00:14', 3, 'Continued high volume of ICMP packets.', 'MITIGATED'), # log_id 14
    (15, 'Ping Flood', '2023-10-26 10:00:15', 3, 'Continued high volume of ICMP packets.', 'MITIGATED'), # log_id 15
    (16, 'Ping Flood', '2023-10-26 10:00:16', 3, 'Continued high volume of ICMP packets.', 'MITIGATED'), # log_id 16
    (17, 'Ping Flood', '2023-10-26 10:00:17', 3, 'Continued high volume of ICMP packets.', 'MITIGATED')  # log_id 17
]
cursor.executemany("INSERT INTO attack_details (log_id, attack_type, detection_time, threat_level_id, description, status) VALUES (?, ?, ?, ?, ?, ?)", attack_details_data)

conn.commit()
print("Database setup complete and sample data inserted.")

Database setup complete and sample data inserted.


### 5. Execute Query 1: Fetch attack details along with network traffic

In [ ]:
# SQL Query 1: Fetch attack details along with network traffic
query1 = """SELECT
    ad.attack_id,
    ad.attack_type,
    ad.detection_time,
    tl.level_name AS threat_level,
    ad.description AS attack_description,
    ad.status,
    nl.timestamp AS log_timestamp,
    nl.source_ip,
    nl.destination_ip,
    nl.protocol,
    nl.port,
    nl.action,
    nl.bytes_sent,
    nl.bytes_received,
    nl.event_type
FROM
    attack_details ad
JOIN
    network_logs nl ON ad.log_id = nl.log_id
JOIN
    threat_levels tl ON ad.threat_level_id = tl.level_id;
"""

# Execute the query and load results into a pandas DataFrame
df_attack_network_traffic = pd.read_sql_query(query1, conn)

# Display the first few rows of the DataFrame
print("DataFrame for Query 1: Attack Details with Network Traffic")
display(df_attack_network_traffic.head())

print(f"\nTotal rows: {len(df_attack_network_traffic)}")

DataFrame for Query 1: Attack Details with Network Traffic


,attack_id,attack_type,detection_time,threat_level,attack_description,status,log_timestamp,source_ip,destination_ip,protocol,port,action,bytes_sent,bytes_received,event_type
0,1,DDoS,2023-10-26 10:00:03,High,Multiple connection attempts from different so...,DETECTED,2023-10-26 10:00:03,192.168.1.12,10.0.0.3,TCP,443.0,DENY,0,0,CONNECTION_ATTEMPT
1,2,Brute Force,2023-10-26 10:00:04,High,Repeated failed login attempts.,DETECTED,2023-10-26 10:00:04,192.168.1.13,10.0.0.4,TCP,22.0,ALERT,256,128,LOGIN_ATTEMPT
2,3,Brute Force,2023-10-26 10:00:06,High,More failed login attempts.,INVESTIGATING,2023-10-26 10:00:06,192.168.1.13,10.0.0.4,TCP,22.0,ALERT,512,256,LOGIN_ATTEMPT
3,4,SQL Injection,2023-10-26 10:00:09,Critical,Attempt to inject malicious SQL code.,DETECTED,2023-10-26 10:00:09,192.168.1.15,10.0.0.6,TCP,3306.0,DENY,0,0,SQL_ATTEMPT
4,5,Ping Flood,2023-10-26 10:00:11,High,High volume of ICMP packets.,MITIGATED,2023-10-26 10:00:11,192.168.1.17,10.0.0.8,ICMP,NaN,ALERT,64,64,PING_SWEEP



Total rows: 10


### 1. Fetch attack details along with network traffic

This query joins the `attack_details` table with the `network_logs` table using `log_id` to provide a comprehensive view of detected attacks and the underlying network events that triggered them.

In [ ]:
print("--- Query 1: Fetch attack details along with network traffic ---")
print("""SELECT
    ad.attack_id,
    ad.attack_type,
    ad.detection_time,
    tl.level_name AS threat_level,
    ad.description AS attack_description,
    ad.status,
    nl.timestamp AS log_timestamp,
    nl.source_ip,
    nl.destination_ip,
    nl.protocol,
    nl.port,
    nl.action,
    nl.bytes_sent,
    nl.bytes_received,
    nl.event_type
FROM
    attack_details ad
JOIN
    network_logs nl ON ad.log_id = nl.log_id
JOIN
    threat_levels tl ON ad.threat_level_id = tl.level_id;
""")

--- Query 1: Fetch attack details along with network traffic ---
SELECT
    ad.attack_id,
    ad.attack_type,
    ad.detection_time,
    tl.level_name AS threat_level,
    ad.description AS attack_description,
    ad.status,
    nl.timestamp AS log_timestamp,
    nl.source_ip,
    nl.destination_ip,
    nl.protocol,
    nl.port,
    nl.action,
    nl.bytes_sent,
    nl.bytes_received,
    nl.event_type
FROM
    attack_details ad
JOIN
    network_logs nl ON ad.log_id = nl.log_id
JOIN
    threat_levels tl ON ad.threat_level_id = tl.level_id;



### 2. Count total attacks by type

This query counts the occurrences of each `attack_type` from the `attack_details` table, providing an overview of the most frequent types of cyber threats.

In [ ]:
print("--- Query 2: Count total attacks by type ---")
print("""SELECT
    attack_type,
    COUNT(attack_id) AS total_attacks
FROM
    attack_details
GROUP BY
    attack_type
ORDER BY
    total_attacks DESC;
""")

--- Query 2: Count total attacks by type ---
SELECT
    attack_type,
    COUNT(attack_id) AS total_attacks
FROM
    attack_details
GROUP BY
    attack_type
ORDER BY
    total_attacks DESC;



### 3. Retrieve high-risk threats based on associated network activity and threat level

This query identifies attacks classified as 'High' or 'Critical' threat levels and also have a significant number of associated network log entries (used as a proxy for 'packet_rate'). It joins `attack_details`, `threat_levels`, and `network_logs` to filter for high-risk scenarios. For 'packet_rate', we're counting the number of `network_logs` entries per attack ID as an indicator of heavy network activity.

In [ ]:
print("--- Query 3: Retrieve high-risk threats based on associated network activity and threat level ---")
print("""SELECT
    ad.attack_id,
    ad.attack_type,
    ad.detection_time,
    tl.level_name AS threat_level,
    COUNT(nl.log_id) AS associated_log_entries_count, -- Proxy for 'packet_rate'
    ad.description AS attack_description,
    ad.status
FROM
    attack_details ad
JOIN
    threat_levels tl ON ad.threat_level_id = tl.level_id
LEFT JOIN
    network_logs nl ON ad.log_id = nl.log_id
WHERE
    tl.level_name IN ('High', 'Critical')
GROUP BY
    ad.attack_id, ad.attack_type, ad.detection_time, tl.level_name, ad.description, ad.status
HAVING
    COUNT(nl.log_id) > 5 -- Example threshold for 'high' packet rate (adjust as needed)
ORDER BY
    associated_log_entries_count DESC, ad.detection_time DESC;
""")

--- Query 3: Retrieve high-risk threats based on associated network activity and threat level ---
SELECT
    ad.attack_id,
    ad.attack_type,
    ad.detection_time,
    tl.level_name AS threat_level,
    COUNT(nl.log_id) AS associated_log_entries_count, -- Proxy for 'packet_rate'
    ad.description AS attack_description,
    ad.status
FROM
    attack_details ad
JOIN
    threat_levels tl ON ad.threat_level_id = tl.level_id
LEFT JOIN
    network_logs nl ON ad.log_id = nl.log_id
WHERE
    tl.level_name IN ('High', 'Critical')
GROUP BY
    ad.attack_id, ad.attack_type, ad.detection_time, tl.level_name, ad.description, ad.status
HAVING
    COUNT(nl.log_id) > 5 -- Example threshold for 'high' packet rate (adjust as needed)
ORDER BY
    associated_log_entries_count DESC, ad.detection_time DESC;



### Relational Database Schema for Cyber Threat Monitoring

This schema includes three tables designed to store information about network activity, detected attacks, and predefined threat levels.

#### 1. `threat_levels`

This is a reference table to categorize the severity of threats.

*   **`level_id`**: Primary key, unique identifier for each threat level.
*   **`level_name`**: Name of the threat level (e.g., 'Low', 'Medium', 'High', 'Critical').
*   **`description`**: A brief explanation of what each threat level signifies.

#### 2. `network_logs`

This table stores detailed records of network traffic and events.

*   **`log_id`**: Primary key, unique identifier for each network log entry.
*   **`timestamp`**: The exact time the network event occurred.
*   **`source_ip`**: The IP address from which the traffic originated.
*   **`destination_ip`**: The IP address to which the traffic was directed.
*   **`protocol`**: The network protocol used (e.g., TCP, UDP, ICMP).
*   **`port`**: The destination port number.
*   **`action`**: The action taken by the network device (e.g., 'ALLOW', 'DENY', 'ALERT').
*   **`bytes_sent`**: The number of bytes sent in the event.
*   **`bytes_received`**: The number of bytes received in the event.
*   **`event_type`**: A general classification of the network event (e.g., 'CONNECTION', 'PACKET_DROP', 'LOGIN_ATTEMPT').

#### 3. `attack_details`

This table stores information about identified cyber attacks, linking them to specific network logs and threat levels.

*   **`attack_id`**: Primary key, unique identifier for each detected attack.
*   **`log_id`**: Foreign key referencing `network_logs.log_id`, linking the attack to a specific network log entry that triggered its detection.
*   **`attack_type`**: The type of attack detected (e.g., 'DDoS', 'SQL Injection', 'Malware Infection', 'Brute Force').
*   **`detection_time`**: The timestamp when the attack was detected.
*   **`threat_level_id`**: Foreign key referencing `threat_levels.level_id`, indicating the severity of the detected attack.
*   **`description`**: A detailed description of the attack, its potential impact, and any observed characteristics.
*   **`status`**: The current status of the attack (e.g., 'DETECTED', 'MITIGATED', 'INVESTIGATING', 'RESOLVED').

In [ ]:
print("--- CREATE TABLE Statements ---")

# Table: threat_levels
print("""CREATE TABLE threat_levels (
    level_id INT PRIMARY KEY,
    level_name VARCHAR(50) NOT NULL UNIQUE,
    description TEXT
);
""")

# Table: network_logs
print("""CREATE TABLE network_logs (
    log_id INT PRIMARY KEY AUTO_INCREMENT,
    timestamp DATETIME NOT NULL,
    source_ip VARCHAR(45),
    destination_ip VARCHAR(45),
    protocol VARCHAR(10),
    port INT,
    action VARCHAR(20),
    bytes_sent INT,
    bytes_received INT,
    event_type VARCHAR(50)
);
""")

# Table: attack_details
print("""CREATE TABLE attack_details (
    attack_id INT PRIMARY KEY AUTO_INCREMENT,
    log_id INT,
    attack_type VARCHAR(100) NOT NULL,
    detection_time DATETIME NOT NULL,
    threat_level_id INT,
    description TEXT,
    status VARCHAR(50),
    FOREIGN KEY (log_id) REFERENCES network_logs(log_id),
    FOREIGN KEY (threat_level_id) REFERENCES threat_levels(level_id)
);
""")

--- CREATE TABLE Statements ---
CREATE TABLE threat_levels (
    level_id INT PRIMARY KEY,
    level_name VARCHAR(50) NOT NULL UNIQUE,
    description TEXT
);

CREATE TABLE network_logs (
    log_id INT PRIMARY KEY AUTO_INCREMENT,
    timestamp DATETIME NOT NULL,
    source_ip VARCHAR(45),
    destination_ip VARCHAR(45),
    protocol VARCHAR(10),
    port INT,
    action VARCHAR(20),
    bytes_sent INT,
    bytes_received INT,
    event_type VARCHAR(50)
);

CREATE TABLE attack_details (
    attack_id INT PRIMARY KEY AUTO_INCREMENT,
    log_id INT,
    attack_type VARCHAR(100) NOT NULL,
    detection_time DATETIME NOT NULL,
    threat_level_id INT,
    description TEXT,
    status VARCHAR(50),
    FOREIGN KEY (log_id) REFERENCES network_logs(log_id),
    FOREIGN KEY (threat_level_id) REFERENCES threat_levels(level_id)
);

